In [1]:
import os
from functools import partial
import json
import csv

import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.metrics import efficiency_gap, mean_median, partisan_gini
from gerrychain.proposals import recom
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [2]:
graph_path = "./texas.json"
graph = Graph.from_json(graph_path)
with open (graph_path, "r") as myfile:
    data=myfile.readlines()
graph_dict = json.loads(data[0])

In [29]:
with open("test.csv", mode="w") as test:
    data_writer = csv.writer(test, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)   
    
    data_writer.writerow(["dem_votes", "rep_votes", "prec_pop", "node_num"])

    for n in graph.nodes:
        data_writer.writerow([graph.nodes[n]["SEN16D"],
                  graph.nodes[n]["SEN16R"],
                  graph.nodes[n]["TOTPOP"],
                  graph.nodes[n]["PrcncID"]])

In [4]:
graph.nodes[0]

{'boundary_node': False,
 'area': 5616315.599709552,
 'CNTYVTD': '10001',
 'VTD': '0001',
 'WHITE': 2053.0,
 'OTHER': 89.0,
 'HISPANIC': 401.0,
 'TOTPOP': 3131.0,
 'VAP': 2341.0,
 'BLACK': 606.0,
 'BLKHISP': 989.0,
 'WVAP': 1677.0,
 'HISPVAP': 217.0,
 'BHVAP': 592.0,
 'BVAP': 378.0,
 'OTHVAP': 72.0,
 'COUNTY': 'Anderson',
 'FIPS': 1,
 'PRES12R': 754,
 'PRES12D': 261,
 'SEN12R': 713,
 'SEN12D': 285,
 'TOTVR12': 1629,
 'TOTTO12': 1026,
 'SEN14R': 441,
 'SEN14D': 110,
 'GOV14R': 424,
 'GOV14D': 130,
 'TOTVR14': 1712,
 'TOTTO14': 576,
 'PRES16D': 262,
 'PRES16R': 742,
 'TOTVR16': 1799,
 'TOTTO16': 1030,
 'USCD': '05',
 'SEND': 3,
 'HD': 8,
 'AREA': 6,
 'PERIM': 15}

In [5]:
TOTPOP = "TOTPOP"
DEMVOTE = "SEN12D"
REPVOTE = "SEN12R"

In [6]:
pop_count = 0

for i in graph.nodes:
    pop_count += graph.nodes[i][TOTPOP]
    
print(pop_count)

25145561.0


In [8]:
%%time
num_dist = 36

# Exercise: Compute exact population from your data.
pop = pop_count

my_updaters = {
    "population": updaters.Tally("TOTPOP", alias="population"),
    "cut_edges": cut_edges,
    "SEN12": Election("SEN08", {"democratic":DEMVOTE,"republican":REPVOTE})
}


new_plan = recursive_tree_part(graph,
                               range(num_dist),
                               pop/num_dist,
                               TOTPOP,
                               0.004,
                               1)
initial_partition = Partition(graph,
                              new_plan,
                              my_updaters)

CPU times: user 27.9 s, sys: 635 ms, total: 28.6 s
Wall time: 29 s


In [9]:
pop_prec = []
reps_prec = []
dems_prec = []

for n in tqdm_notebook(graph.nodes):
    pop_prec.append(graph.nodes[n][TOTPOP])
    reps_prec.append(graph.nodes[n][REPVOTE])
    dems_prec.append(graph.nodes[n][DEMVOTE])

In [10]:
(dems_prec[0] - reps_prec[0]) / pop_prec[0]

-0.38460756552620307

In [10]:
ascending_DPC = []

for i in range(len(pop_prec)):
    if pop_prec[i] != 0:
        ascending_DPC.append([(dems_prec[i] - reps_prec[i]) / pop_prec[i], i])

In [11]:
len(ascending_DPC)

8634

In [12]:
ascending_DPC[:10]

[[-0.13669754072181411, 0],
 [-0.21314102564102563, 1],
 [-0.2723669309173273, 2],
 [-0.24219841639496972, 3],
 [-0.25840336134453784, 4],
 [0.04288407163053723, 5],
 [0.10842237586423632, 6],
 [0.03633060853769301, 7],
 [-0.15886524822695036, 8],
 [-0.032203864463735646, 9]]

In [13]:
ascending_DPC.sort()

In [14]:
ascending_DPC[:10]

[[-4.714285714285714, 7829],
 [-3.227777777777778, 6022],
 [-1.9898477157360406, 6023],
 [-1.0338983050847457, 8628],
 [-1.0, 4560],
 [-1.0, 4599],
 [-1.0, 4943],
 [-0.8571428571428571, 5060],
 [-0.8333333333333334, 4709],
 [-0.8085106382978723, 5288]]

In [15]:
descending_DPC = ascending_DPC.copy()
descending_DPC.reverse()

In [16]:
descending_DPC[:10]

[[4.157894736842105, 3418],
 [1.75, 2988],
 [1.5, 2994],
 [1.5, 816],
 [1.0, 7413],
 [1.0, 4992],
 [1.0, 3062],
 [1.0, 3027],
 [1.0, 968],
 [0.92, 6785]]

In [17]:
ascending_DPC[:10]

[[-4.714285714285714, 7829],
 [-3.227777777777778, 6022],
 [-1.9898477157360406, 6023],
 [-1.0338983050847457, 8628],
 [-1.0, 4560],
 [-1.0, 4599],
 [-1.0, 4943],
 [-0.8571428571428571, 5060],
 [-0.8333333333333334, 4709],
 [-0.8085106382978723, 5288]]

In [18]:
I = pop_count/36
print(I)

698487.8055555555


In [19]:
pop_counter = 0
dem_qdist_1 = []
dem_qdist_2 = []

for j in descending_DPC:
    if pop_counter <= I:
        dem_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]][TOTPOP]
    elif pop_counter > I and pop_counter <= 2*I:
        dem_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]][TOTPOP]
    else:
        break

In [20]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in dem_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n][DEMVOTE]
    rep_votes_qdist_1 += graph.nodes[n][REPVOTE]
    total_pop_qdist_1 += graph.nodes[n][TOTPOP]
    
for n in dem_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n][DEMVOTE]
    rep_votes_qdist_2 += graph.nodes[n][REPVOTE]
    total_pop_qdist_2 += graph.nodes[n][TOTPOP]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.0731084833331031 0.1612265854933206
0.1109692922846619


In [23]:
pop_counter = 0
rep_qdist_1 = []
rep_qdist_2 = []

for j in ascending_DPC:
    if pop_counter <= I:
        rep_qdist_1.append(j[1])
        pop_counter += graph.nodes[j[1]][TOTPOP]
    elif pop_counter > I and pop_counter <= 2*I:
        rep_qdist_2.append(j[1])
        pop_counter += graph.nodes[j[1]][TOTPOP]
    else:
        break

In [24]:
dem_votes_qdist_1 = 0
rep_votes_qdist_1 = 0
total_pop_qdist_1 = 0
dem_votes_qdist_2 = 0
rep_votes_qdist_2 = 0
total_pop_qdist_2 = 0

for n in rep_qdist_1:
    dem_votes_qdist_1 += graph.nodes[n][DEMVOTE]
    rep_votes_qdist_1 += graph.nodes[n][REPVOTE]
    total_pop_qdist_1 += graph.nodes[n][TOTPOP]
    
for n in rep_qdist_2:
    dem_votes_qdist_2 += graph.nodes[n][DEMVOTE]
    rep_votes_qdist_2 += graph.nodes[n][REPVOTE]
    total_pop_qdist_2 += graph.nodes[n][TOTPOP]
    
votes_qdist_1 = rep_votes_qdist_1 + dem_votes_qdist_1
votes_qdist_2 = rep_votes_qdist_2 + dem_votes_qdist_2

rep_vs_qdist_1 = rep_votes_qdist_1 / votes_qdist_1
rep_vs_qdist_2 = rep_votes_qdist_2 / votes_qdist_2

print(rep_vs_qdist_1, rep_vs_qdist_2)

overall_rep_vs = (rep_votes_qdist_1 + rep_votes_qdist_2) / (votes_qdist_1 + votes_qdist_2)
print(overall_rep_vs)

0.8410830359458935 0.8206926608170143
0.8316152871707079
